In [1]:
import pandas as pd

from pathlib import Path

from star_parser import StarParser, StarTabDf

In [2]:
work_folder = Path(r"/mnt/DATA/andrea/2021_JBP3_reprocess/relion4")
star_in=work_folder/"J259_extracted_particles.star"
star_in.exists()

True

In [3]:
parser = StarParser(star_in)
data = parser.parse()


In [4]:
ptcls = data["data_particles"]
ptcls.keep_only_columns(['MicrographName','CoordinateX','CoordinateY'], store=True)

,MicrographName,CoordinateX,CoordinateY
0,J2/motioncorrected/000012806598761942185_FoilH...,2973,1581
1,J2/motioncorrected/000012806598761942185_FoilH...,1797,2179
2,J2/motioncorrected/000012806598761942185_FoilH...,337,3403
3,J2/motioncorrected/000012806598761942185_FoilH...,3939,3193
4,J2/motioncorrected/000012806598761942185_FoilH...,3748,3011
...,...,...,...
238892,J2/motioncorrected/018435645858845947095_FoilH...,4129,1869
238893,J2/motioncorrected/018435645858845947095_FoilH...,4683,2475
238894,J2/motioncorrected/018435645858845947095_FoilH...,901,1017
238895,J2/motioncorrected/018435645858845947095_FoilH...,5262,1751


In [5]:
prefix = "J2/motioncorrected/"
ptcls.remove_string_from_column_name(prefix, 'MicrographName', store=True)

,MicrographName,CoordinateX,CoordinateY
0,000012806598761942185_FoilHole_13153438_Data_1...,2973,1581
1,000012806598761942185_FoilHole_13153438_Data_1...,1797,2179
2,000012806598761942185_FoilHole_13153438_Data_1...,337,3403
3,000012806598761942185_FoilHole_13153438_Data_1...,3939,3193
4,000012806598761942185_FoilHole_13153438_Data_1...,3748,3011
...,...,...,...
238892,018435645858845947095_FoilHole_13153416_Data_1...,4129,1869
238893,018435645858845947095_FoilHole_13153416_Data_1...,4683,2475
238894,018435645858845947095_FoilHole_13153416_Data_1...,901,1017
238895,018435645858845947095_FoilHole_13153416_Data_1...,5262,1751


In [6]:
ptcls.trim_column_values('MicrographName', start=len('005568354855995828403_'), store=True)


,MicrographName,CoordinateX,CoordinateY
0,FoilHole_13153438_Data_13141691_13141693_20200...,2973,1581
1,FoilHole_13153438_Data_13141691_13141693_20200...,1797,2179
2,FoilHole_13153438_Data_13141691_13141693_20200...,337,3403
3,FoilHole_13153438_Data_13141691_13141693_20200...,3939,3193
4,FoilHole_13153438_Data_13141691_13141693_20200...,3748,3011
...,...,...,...
238892,FoilHole_13153416_Data_13141691_13141693_20200...,4129,1869
238893,FoilHole_13153416_Data_13141691_13141693_20200...,4683,2475
238894,FoilHole_13153416_Data_13141691_13141693_20200...,901,1017
238895,FoilHole_13153416_Data_13141691_13141693_20200...,5262,1751


In [7]:
[p for p in ptcls.to_df()['MicrographName'] if not p.startswith('FoilHole')]

[]

In [8]:
remove_cryosparc_suffix = True
if remove_cryosparc_suffix:
    ptcls.remove_string_from_column_name('_patch_aligned_doseweighted', 'MicrographName', store=True)
    ptcls.to_df()['MicrographName'][0]

In [10]:
split = ptcls.to_df().groupby('MicrographName')
split_dir = work_folder/'J259_particles'
if not split_dir.exists():
    split_dir.mkdir()
for fname, data in split:
    x = StarTabDf(data)
    fname = fname.replace('.mrc','.star')
    with open(split_dir/fname, 'w') as f:
        f.write(x.to_star())


In [ ]:
#if we want to train cryolo, we need to remove the data_ and loop_ lines from the header
cryolo_train = False
starfiles = split_dir.glob("*.star")
if cryolo_train:
    for f in starfiles:
        text = f.read_text().split('\n')
        for index, line in enumerate(text):
            if line.startswith('_rln'):
                break
        f.write_text('\n'.join(text[index:]))

